In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
# from model import BertForTokenClassification #DoubleBorderModel
# from utils import random_split_train_valid_test
import torch
import json
from transformers import BertTokenizerFast
# from ngram_train import *
from tqdm import tqdm
import datetime
import random
import numpy as np
import time
from tqdm import tqdm
import torch.nn as nn
from lrb.model import *
from ner.model import *
id2label = ['O','B','I','E','S']
label2id = {label:i for i,label in enumerate(id2label)}
MODEL_NAME = 'hfl/chinese-roberta-wwm-ext'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ner_model = NERModel.from_pretrained(MODEL_NAME,num_labels=len(id2label))
ner_ckpt = "/data2/private/wzt/term11_22/term/ner/ckpts/strict/ner_strict_epoch_1_split_19.pt"
ner_model.load_state_dict(torch.load(ner_ckpt))
ner_model = nn.DataParallel(ner_model)
ner_model.to(device)

MAX_LENGTH = 128
ngram_model =  LRBModel.from_pretrained(MODEL_NAME,num_labels=2,max_length= MAX_LENGTH)
ngram_ckpt = "/data2/private/wzt/term11_22/term/lrb/ckpts/strict/lrb_jian12_strict_epoch_1_split_19.pt"
ngram_model.load_state_dict(torch.load(ngram_ckpt))
ngram_model.to(device)
tk = BertTokenizerFast.from_pretrained(MODEL_NAME)
import os
from attn.model import TermModel
import json
import torch
import torch.nn as nn
from transformers import BertTokenizerFast
import torch.utils.data as td

sam = 0
uniq = 'wzt_alladdc'
epoch = 1

class Args:
    model_name = "hfl/chinese-roberta-wwm-ext"

    finetuned_ckpt = f"/data2/private/wzt/term11_22/term/attn/ckpts/all_no23c_neg_5sam1/all_no23c_neg_5sam1_epoch8_split_19.pt"
    context_size = 3
    
    use_two_classifier = 0

args = Args()

tk = BertTokenizerFast.from_pretrained(args.model_name)

attn_model = TermModel.from_pretrained(args.model_name, num_labels=2)
setattr(attn_model.config, 'args', args)

initialize_from_existing_ckpt = not (args.finetuned_ckpt == "" or args.finetuned_ckpt is None)

if initialize_from_existing_ckpt:
    print("loading finetuned ckpt")
    attn_model.load_state_dict(torch.load(args.finetuned_ckpt))
setattr(attn_model, 'tk', tk)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
attn_model.to(device)
pass

In [ ]:
sent = '牛奶是从奶牛的乳房中挤压出来的'
encoded_dict = tk([list(sent),],
                        padding='max_length',
                        truncation=True,
                        max_length=MAX_LENGTH,
                        is_split_into_words=True,
                        add_special_tokens=True)
# print(encoded_dict)
input_ids = torch.tensor(encoded_dict['input_ids'], dtype=torch.long) 
attention_mask = torch.tensor(encoded_dict['attention_mask'], dtype=torch.long) 
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

import traceback
# NGRAM
try:
    ngram_outputs = ngram_model(input_ids = input_ids, 
        attention_mask = attention_mask)
except:
    traceback.print_exc()
bs,seqlen,hs = ngram_outputs[0].shape
logits = torch.softmax(ngram_outputs[0],dim=-1)[:,:,1].detach().cpu().numpy().reshape(bs,2,seqlen//2)
logits = np.vstack((logits[:,0,:-1],logits[:,1,1:]))[np.newaxis,:]
for i in range(len(sent)):
    lprob = format(logits[0,0,i],'.4f')
    rprob = format(logits[0,1,i],'.4f')
    print(sent[i],lprob,rprob)

In [ ]:
b2z={True:'真',False:'假'}
def B2Q(uchar):
    """单个字符 半角转全角"""
    inside_code = ord(uchar)
    if inside_code < 0x0020 or inside_code > 0x7e: # 不是半角字符就返回原来的字符
        return uchar 
    if inside_code == 0x0020: # 除了空格其他的全角半角的公式为: 半角 = 全角 - 0xfee0
        inside_code = 0x3000
    else:
        inside_code += 0xfee0
    return chr(inside_code)
def stringB2Q(s):
    """字符串 半角转全角"""
    return ''.join([B2Q(uchar) for uchar in s])
lt = [0.8,0.9,0.97]
rt = [0.8,0.9,0.95]
ngram_model.to(device)
ner_model.eval()
ngram_model.eval()
tmp = 'DFA图'

all_sents_output=[]
total_sents_cnt = 0
lrb_sents_cnt = 0
ner_sents_cnt = 0
def see_sent(test_sent,tqdm_disable=False):
    for sent in tqdm(test_sent,disable=tqdm_disable):
        # print(sent)
        # print(sent)
        sent = list(sent)
        # print(sent)
        # NER
        encoded_dict = tk([sent],
                        padding='max_length',
                        truncation=True,
                        max_length=MAX_LENGTH,
                        is_split_into_words=True,
                        add_special_tokens=False)

        input_ids = torch.tensor(encoded_dict['input_ids'], dtype=torch.long)
        attention_mask = torch.tensor(encoded_dict['attention_mask'], dtype=torch.long)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        ner_output = ner_model(input_ids, attention_mask=attention_mask,pad_mask = attention_mask)[0].squeeze(0)
        # crf_pred = ner_model.module.crf.decode(ner_output.unsqueeze(0),mask = attention_mask.bool())[0]

        encoded_dict = tk([sent],
                        padding='max_length',
                        truncation=True,
                        max_length=MAX_LENGTH,
                        is_split_into_words=True,
                        add_special_tokens=True)
        # print(encoded_dict)
        input_ids = torch.tensor(encoded_dict['input_ids'], dtype=torch.long) 
        attention_mask = torch.tensor(encoded_dict['attention_mask'], dtype=torch.long) 
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        # NGRAM
        ngram_outputs = ngram_model(input_ids = input_ids, 
                attention_mask = attention_mask)
        bs,seqlen,hs = ngram_outputs[0].shape
        logits = torch.softmax(ngram_outputs[0],dim=-1)[:,:,1].detach().cpu().numpy().reshape(bs,2,seqlen//2)
        logits = np.vstack((logits[:,0,:-1],logits[:,1,1:]))[np.newaxis,:]
        sent_len = min(len(sent),len(logits[0][0]))
        lrs = ['']*sent_len
        lrcnt = 0

        lpairs = []
        rpairs = []
        for i in range(sent_len):
                lprob = logits[0][0][i]
                rprob = logits[0][1][i]
                

                if lprob>lt[0]:
                    rthres = 0
                    if lprob>lt[2]:
                        rthres = rt[0]
                    elif lprob>lt[1]:
                        rthres = rt[1]
                    else:
                        rthres = rt[2]

                    for j in range(i+2,sent_len):
                        if logits[0][1][j]>rthres:
                            lpairs.append((i,j))
                            lrs[i]+='左'+stringB2Q(str(lrcnt))
                            lrs[j] +='右'+stringB2Q(str(lrcnt))
                            lrcnt+=1
                            break
        r_lrcnt = 0
        r_lrs = ['']*sent_len
        for i in range(sent_len-1,-1,-1):
                # print(sent[i])
                lprob = logits[0][0][i]
                rprob = logits[0][1][i]
                

                if rprob>rt[0]:
                    lthres = 0
                    if rprob>rt[2]:
                        lthres = lt[0]
                    elif rprob>rt[1]:
                        lthres = lt[1]
                    else:
                        lthres = lt[2]

                    for j in range(i-2,-1,-1):
                        if logits[0][0][j]>lthres:
                            if (j,i) not in lpairs:
                                rpairs.append((j,i))
                                r_lrs[j] +='左'+stringB2Q(str(r_lrcnt))
                                r_lrs[i] +='右'+stringB2Q(str(r_lrcnt))
                                r_lrcnt += 1
                            break
        # print(lpairs,file=f)
        # print(rpairs,file=f)
        print_output = []
        sent_bieos = []
        lrbsee_output = []
        nersee_output = []
        for i in range(sent_len):
                # print(sent[i])
                lprob = logits[0][0][i]
                rprob = logits[0][1][i]
                

                lr = lrs[i]
                pad_len = 9-len(lr)
                if pad_len>0:
                    # lr += B2Q(' ')*pad_len
                    lr += ' '*pad_len
                
                r_lr = r_lrs[i]
                pad_len = 3-len(r_lr)
                if pad_len>0:
                    r_lr += ' '*pad_len
                    # r_lr += B2Q(' ')*pad_len
                
                lprob = format(logits[0,0,i],'.3f')
                rprob = format(logits[0,1,i],'.3f')
                bieos = ner_output[i]
                bieos = torch.softmax(bieos,dim=-1)
                top2_prob = torch.topk(bieos,2)[1].detach().cpu().numpy()
                top1 = ''
                top2 = ''
                suffix = ''
                top1 = top2_prob[0]
                prob1 = format(bieos[top1],'.3f')
                suffix += f'{id2label[top1]} = {prob1}'
                sent_bieos.append([id2label[top1]])

                top2 = top2_prob[1]
                prob2 = format(bieos[top2],'.3f')
                thres = 0.6
                if id2label[top1]=='E' and id2label[top2]=='I':
                        thres = 0.5
                # else:
                if bieos[top2] / bieos[top1] > thres:
                    sent_bieos[-1].append(id2label[top2])
                    suffix += f', {id2label[top2]} = {prob2}'

                bieos = [id2label[j]+'=='+format(item,'.3f') for j,item in enumerate(bieos)]
                bieos = ','.join(bieos)
                char = B2Q(sent[i])
                lrbsee_output.append(' '.join([char,f'(左:{lprob},右:{rprob})',lr,r_lr]))
                nersee_output.append(' '.join([char,' ',suffix]))
    return lrbsee_output,nersee_output



In [ ]:
sent = "学生和当局发生多次严重冲突。"
res = see_sent([sent])
for item in res:
    for line in item:
        print(line)
res = attn_model.inference_one_sent(sent, top_k = -1, verbose=False,sm=True,smt=2)
for item in res:
    ngram = ''.join(item[0][0])
    prob = format(item[1],'.3f')
    print(f'{ngram},{prob}')

In [ ]:
data = []
with open('/home/wangzhitong/data2/strict_sents.txt','r') as f:
    for line in tqdm(f):
        data.append(line.strip())

In [ ]:
lrb_results = []
ner_results = []
attn_results = []
for sent in tqdm(data):
    lrb,ner = see_sent([sent],True)
    lrb_results.append(lrb)
    ner_results.append(ner)
    attn_results.append(attn_model.inference_one_sent(sent, top_k = -1, verbose=False,sm=True,smt=2))
attn_results = [[ [item[0][0],item[0][1].numpy().tolist(),format(item[1],'.4f')] for item in sent ] for sent in attn_results]

In [ ]:
cnt=0

ner_results = []
ner_hl = []
for sent_idx,sent in enumerate(tqdm(data)):
    # print()
    sent = list(sent)
    encoded_dict = tk([sent],
                    padding='max_length',
                    truncation=True,
                    max_length=MAX_LENGTH,
                    is_split_into_words=True,
                    add_special_tokens=True)

    input_ids = torch.tensor(encoded_dict['input_ids'], dtype=torch.long)
    attention_mask = torch.tensor(encoded_dict['attention_mask'], dtype=torch.long)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    ner_output = ner_model(input_ids, attention_mask=attention_mask,pad_mask = attention_mask)[0].squeeze(0)
    # print(ner_output)
    sent_len = min(len(sent),MAX_LENGTH-2)
    sent_bieos = []
    for i in range(1,sent_len+1):
        bieos = ner_output[i]
        bieos = torch.softmax(bieos,dim=-1)
        top2_prob = torch.topk(bieos,2)[1].detach().cpu().numpy()
        top1 = ''
        top2 = ''
        top1 = top2_prob[0]
        prob1 = format(bieos[top1],'.3f')
        sent_bieos.append([id2label[top1]])
        top2 = top2_prob[1]
        prob2 = format(bieos[top2],'.3f')
        thres = 0.6
        if id2label[top1]=='E' and id2label[top2]=='I':
            thres = 0.5
        # else:
        if bieos[top2] / bieos[top1] > thres:
            sent_bieos[-1].append(id2label[top2])
    sent_hl = []
    for i,label in enumerate(sent_bieos):
        if 'B' in label:
            for j in range(i+1,len(sent_bieos)):
                if 'E' in sent_bieos[j]:
                    sent_hl.append([i,j])
                    # ner_hl
                    break
                elif 'I' in sent_bieos[j]:
                    continue
                else:
                    break
    ner_hl.append(sent_hl)
    ner_results.append(ner_output.detach().cpu().numpy())
    # for i,label in enumerate(sent_bieos):
    #     print(sent[i],label)

In [207]:
lt = [0.8,0.9,0.97]
rt = [0.8,0.9,0.95]
lrb_results = []
lrb_logits = []
for sent_idx,sent in enumerate(tqdm(data)):
    # print()
    sent = list(sent)
    encoded_dict = tk([sent],
                    padding='max_length',
                    truncation=True,
                    max_length=MAX_LENGTH,
                    is_split_into_words=True,
                    add_special_tokens=True)

    input_ids = torch.tensor(encoded_dict['input_ids'], dtype=torch.long)
    attention_mask = torch.tensor(encoded_dict['attention_mask'], dtype=torch.long)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    ngram_outputs = ngram_model(input_ids = input_ids, 
                attention_mask = attention_mask)
    bs,seqlen,hs = ngram_outputs[0].shape
    logits = torch.softmax(ngram_outputs[0],dim=-1)[:,:,1].detach().cpu().numpy().reshape(bs,2,seqlen//2)
    logits = np.vstack((logits[:,0,:-1],logits[:,1,1:]))[np.newaxis,:]
    sent_len = min(len(sent),len(logits[0][0]))
    lrs = ['']*sent_len
    lrcnt = 0

    lpairs = []
    rpairs = []
    lrb_table = [[0,0] for _ in range(sent_len)]
    for i in range(sent_len):
            lprob = logits[0][0][i]
            rprob = logits[0][1][i]
            if lprob>lt[0]:
                rthres = 0
                if lprob>lt[2]:
                    rthres = rt[0]
                elif lprob>lt[1]:
                    rthres = rt[1]
                else:
                    rthres = rt[2]

                for j in range(i+1,sent_len):
                    if logits[0][1][j]>rthres:
                        lpairs.append((i,j))
                        lrb_table[i][0] = 1
                        lrb_table[j][1] = 1
                        break
    r_lrcnt = 0
    r_lrs = ['']*sent_len
    for i in range(sent_len-1,-1,-1):
            # print(sent[i])
            lprob = logits[0][0][i]
            rprob = logits[0][1][i]
            

            if rprob>rt[0]:
                lthres = 0
                if rprob>rt[2]:
                    lthres = lt[0]
                elif rprob>rt[1]:
                    lthres = lt[1]
                else:
                    lthres = lt[2]

                for j in range(i-2,-1,-1):
                    if logits[0][0][j]>lthres:
                        if (j,i) not in lpairs:
                            rpairs.append((j,i))
                            lrb_table[j][0] = 1
                            lrb_table[i][1] = 1
                        break
    final_pairs = []
    for pair in lpairs+rpairs:
        l = pair[0]
        if l < sent_len-1:
            if lrb_table[l+1][1]==0 and lrb_table[l+2][1]==0:
                continue
        final_pairs.append(pair)
    lrb_table = [[0,0] for _ in range(sent_len)]
    for pair in final_pairs:
        lrb_table[pair[0]][0] = 1
        lrb_table[pair[1]][1] = 1
    
    lrb_results.append(lrb_table)
    lrb_logits.append(logits)
    # for i in range(sent_len):
    #         # print(sent[i])
    #         lprob = logits[0][0][i]
    #         rprob = logits[0][1][i]
            
    #         lprob = format(logits[0,0,i],'.3f')
    #         rprob = format(logits[0,1,i],'.3f')

    #         lr = ''
    #         lr += '左' if lrb_table[i][0]==1 else ''
    #         lr += '右' if lrb_table[i][1]==1 else ''

    #         print(sent[i],lprob,rprob,lr)
    # break


100%|██████████| 10000/10000 [02:44<00:00, 60.77it/s]


In [ ]:
attn_results = []
for sent in tqdm(data):
    attn_results.append(attn_model.inference_one_sent(list(sent), top_k = -1, verbose=False,sm=True,smt=2))
attn_results = [[ [''.join(item[0][0]),(item[0][1].numpy()-1).tolist(),format(item[1],'.4f')] for item in sent ] for sent in attn_results]

In [ ]:
# attn_results = [[ [''.join(item[0]),item[1],item[2]] for item in sent ] for sent in attn_results]
attn_results[0]

In [204]:
attn_dict = {}
for sent_idx,sent in enumerate(tqdm(attn_results)):
    # print(sent)
    for item in sent:
        if item[0] not in attn_dict:
            attn_dict[item[0]] = []
        attn_dict[item[0]].append([sent_idx]+item[1:])
    # break
attn_dict

100%|██████████| 10000/10000 [00:18<00:00, 544.73it/s]


{'日本': [[0, [7, 8], '0.9335'],
  [9, [109, 110], '0.7428'],
  [34, [30, 31], '0.8983'],
  [34, [86, 87], '0.7908'],
  [48, [16, 17], '0.9602'],
  [48, [38, 39], '0.0019'],
  [77, [7, 8], '0.0085'],
  [77, [19, 20], '0.0023'],
  [86, [20, 21], '0.8356'],
  [223, [36, 37], '0.9959'],
  [228, [2, 3], '0.8377'],
  [235, [9, 10], '0.8910'],
  [240, [11, 12], '0.9311'],
  [240, [80, 81], '0.9150'],
  [240, [57, 58], '0.9054'],
  [265, [21, 22], '0.8977'],
  [314, [33, 34], '0.8976'],
  [357, [15, 16], '0.7980'],
  [470, [0, 1], '0.7184'],
  [475, [7, 8], '0.1067'],
  [475, [18, 19], '0.0062'],
  [533, [0, 1], '0.8337'],
  [552, [35, 36], '0.0110'],
  [560, [1, 2], '0.8873'],
  [714, [0, 1], '0.0538'],
  [761, [51, 52], '0.1149'],
  [794, [34, 35], '0.8338'],
  [794, [0, 1], '0.2335'],
  [869, [21, 22], '0.8656'],
  [923, [19, 20], '0.8976'],
  [966, [17, 18], '0.9390'],
  [973, [7, 8], '0.0298'],
  [1041, [10, 11], '0.8908'],
  [1109, [4, 5], '0.8668'],
  [1151, [47, 48], '0.8679'],
  [1200,

In [ ]:
save_json = {'lrb':lrb_results,'ner':ner_results,'attn':attn_results}
with open('/home/wangzhitong/data2/strict_sents_results.json','w') as f:
    json.dump(save_json,f,ensure_ascii=False,indent=2)

In [ ]:
# ner_hl = {item[0]:item[1] for item in ner_hl}
ner_hl = [set(item) for item in ner_hl]
ner_hl

In [ ]:
hl = []
for sent_idx,sent in enumerate(tqdm(attn_results)):
    for term in sent:
        if term[1][1]<MAX_LENGTH-2:
            if float(term[2]) > 0.5:
                hl.append([term[0],sent_idx,term[1],term[2],'attn'])
            elif term[1] in ner_hl[sent_idx]:
                hl.append([term[0],sent_idx,term[1],term[2],'ner'])
# hl

In [ ]:
new_hl = []
out_hl = []
error_cnt = 0
for item in tqdm(hl):
    # if item[]
    sent_idx = item[1]
    term_idx = item[2]
    try:
        if lrb_results[sent_idx][term_idx[0]][0]==0 or lrb_results[sent_idx][term_idx[1]][1]==0:
            out_hl.append(item)
        else:
            new_hl.append(item)
    except:
        error_cnt+=1
print(error_cnt)
len(new_hl)

In [ ]:
def calc_ner_prob(sent_idx,term_idx):
    bidx = term_idx[0]+1
    eidx = term_idx[1]+1
    sent_res = ner_results[sent_idx]
    probs = []
    probs.append(sent_res[bidx][1])
    for i in range(bidx+1,eidx):
        probs.append(sent_res[i][2])
    probs.append(sent_res[eidx][3])
    prob = sum(probs)/len(probs)
    prob = prob**(1/4)
    return prob
calc_ner_prob(hl[0][1],hl[0][2])

In [ ]:
for item in tqdm(new_hl):
    item.append(calc_ner_prob(item[1],item[2]))
    item.append(item[-1]+float(item[-3]))
new_hl

In [218]:
vhr = []
hr = []
r = []
extracted_terms = []
extract_dict = {}
for item in tqdm(new_hl):
    if item[-1]>1.5:
        vhr.append(item)
    elif item[-1]>0.9:
        hr.append(item)
    elif item[-1]>0.6:
        r.append(item)
    else:
        continue
    extracted_terms.append(item[0])
    if item[0] not in extract_dict:
        extract_dict[item[0]] = []
    extract_dict[item[0]].append(item)
tmp = set([ item for item in extracted_terms if not have_pat(item) and len(item)>2])
len(tmp)

100%|██████████| 50342/50342 [00:00<00:00, 574147.55it/s]


25339

In [219]:
print(len(vhr)/(len(vhr)+len(hr)+len(r)))
print(len(hr)/(len(vhr)+len(hr)+len(r)))
print(len(r)/(len(vhr)+len(hr)+len(r)))

0.5880495023936751
0.3761744899783477
0.0357760076279772


In [192]:
r3 = [item for item in r if len(item[0])>2]

In [198]:
# vhr[0]
tmp = random.choice(r3)
sent = data[tmp[1]]
print(tmp[0])
print(sent)
print('attn概率：',tmp[3])
print('ner概率：',format(tmp[5],'.4f'))
print('总概率',format(tmp[6],'.4f'))

评议部
1906年4月，吴玉章加入同盟会，与但懋辛同被选为评议部评议员。
attn概率： 0.5952
ner概率： 0.2965
总概率 0.8917


In [ ]:
inter = tmp.intersection(gold_terms)
precision = len(inter)/len(tmp)
recall = len(inter)/len(gold_terms)
f1 = 2*precision*recall/(precision+recall)
print(precision,recall,f1)
missed_terms = gold_terms - tmp

In [199]:
missed_in_terms = [item for item in missed_terms if item in out_terms]
len(missed_in_terms)

320

In [226]:
right_out = [item for item in out_terms if item not in gold_terms]
len(right_out)

11480

In [227]:
right_out

['广播',
 '水渠',
 '广东十虎',
 '嗣德',
 '本质',
 '多维',
 '椭圆',
 '五四',
 '东帝汶藉',
 '尼僧',
 '肌肉注射猪瘟弱毒疫苗',
 '赞助',
 '科维托娃',
 '洪呼',
 '中华人民共和国电子签名法',
 '《特攻联盟》',
 '顶员',
 '抑癌基因',
 '威斯康辛大学麦迪逊分校东亚语文',
 '26个重建计划市区重建项目受',
 '常见问题',
 '清洁',
 '数理资优班专题研究课程',
 '威廉',
 '辩证神学家',
 '驻英国大使',
 '完中167中学',
 '短径',
 '斯蒂格尔',
 '伊比利亚',
 '注生娘娘',
 '蒙古',
 '氢融合反映',
 '英国顾问公司',
 '中国基督教教士',
 '天保',
 '上比利牛斯省',
 '主题',
 '摩根·费雪',
 '私烟',
 '星际联邦领域',
 '路易十八',
 '冷核聚变',
 '一所缘境',
 '中国听障',
 '学术会议',
 '[UNK]金三角[UNK]',
 '库尔斯克州',
 '加利福尼亚被美国占领',
 '海峡群岛国家公园',
 '安东尼乌',
 '伊什塔尔城门及游行大街',
 '日本职棒',
 '1961-62赛季芝加哥包装',
 '于芝加哥大学',
 '全球华人新秀大赛',
 '盐腌',
 '自动整理',
 '读者',
 '舌尖低语',
 '平潭海峡',
 '司法界',
 '香港执业律师',
 '底盘',
 '替补成员',
 '克拉伦斯',
 '离子风暴',
 '洪堡08',
 '公正国小',
 '新宿中心大厦地下',
 '丝毯',
 '辽沈战役',
 '四合院式',
 '亚利桑那州',
 '枪支暴力',
 '玉',
 '冷战时期',
 '丝',
 '满目疮痍',
 '圣劳伦斯',
 '乌克兰语',
 '江西节度使',
 '最佳民谣音乐专辑国家',
 '同化',
 '镇',
 '亚里士多德与埃德蒙·伯克',
 '天启',
 '总谐波失真',
 '主观',
 '个人理财',
 '区域苏维埃议会',
 '、爱丽丝',
 '城池',
 '亲缘关系',
 '大竹',
 'namlichdieb',
 '[UNK]寮',
 '大寮交流道',
 '青州州府',
 '充满希望',
 '英国怡和洋行',
 'blogspot.com

In [216]:
p=  35.3
r = 96.9
f1 = (2*p*r)/(p+r)
f1

51.748411497730714

In [222]:
tar = '环境保护'
attn_dict[tar]

[[3769, [10, 13], '0.0352'],
 [6378, [66, 69], '0.6281'],
 [7003, [42, 45], '0.9858']]

In [212]:
lrb_logits[0].shape

(1, 2, 126)

In [225]:
sent_idx = 7003
for char,lrtable,llogit,rlogit in zip(data[sent_idx],lrb_results[sent_idx],lrb_logits[sent_idx][0][0],lrb_logits[sent_idx][0][1]):
    lr = ''
    if lrtable[0]==1:
        lr+='左'
    if lrtable[1]==1:
        lr+='右'
    print(char,format(llogit,'.3f'),format(rlogit,'.3f'),lr)

曾 1.000 0.083 
经 0.001 0.084 
与 0.000 0.087 
前 0.000 0.269 
中 1.000 0.002 左
国 0.000 0.994 右
国 1.000 0.643 左
家 0.054 1.000 右
主 0.999 0.001 左
席 0.000 1.000 右
江 1.000 0.000 左
泽 0.918 0.534 左
民 0.000 1.000 右
合 0.999 0.000 
作 0.000 0.618 
保 1.000 0.003 左
护 0.000 1.000 右
云 1.000 0.000 左
南 0.626 0.999 右
省 0.055 0.995 右
境 0.820 0.000 左
内 0.384 0.962 右
的 0.000 0.000 
虎 1.000 0.521 左
跳 0.836 0.026 左
峡 0.391 1.000 右
， 0.030 0.000 
也 0.012 0.206 
捐 0.921 0.029 
了 0.000 0.262 
价 1.000 0.108 左
值 0.000 0.846 右
1 1.000 0.006 左
亿 0.592 1.000 右
美 0.973 0.001 左
元 0.621 1.000 右
的 0.000 0.000 
高 1.000 0.365 左
盛 0.157 0.948 右
股 0.987 0.319 左
票 0.192 1.000 右
给 0.000 0.000 
环 1.000 0.288 
境 0.556 0.593 
保 0.978 0.057 左
护 0.456 1.000 右
的 0.002 0.000 
教 0.970 0.421 
育 0.477 0.780 
。 0.005 0.001 


In [201]:
print(missed_in_terms)

['别尔哥罗德州', '日本帝国', '北卡罗莱纳', '京西宾馆', '爱因斯坦', '卡斯蒂利亚-莱昂', '天津工商学院', '国际足协', '中央研究院', '阿尔弗雷德', '乌兰乌德', '小行星6', '洲际弹道导弹', '宣誓效忠', '国际影展', '体细胞', '蓄意杀害', '中华人民共和国政治人物', '长臂天牛', '金融机构', '哈尔科夫州', '二战期间', '特鲁埃尔省', '防弹少年团', '机械工程', '威斯康辛', '布列塔尼公爵', '人民解放军', '第25届', '宾夕法尼亚', '公园大道', '说唱歌手', '日耳曼语', '组织暴力团', '用心良苦', '庆应义塾大学', '台湾大学', '新南威尔士州', '海峡群岛', '呼和浩特', '南京西路', '电子音乐', '政治地理学', '莱克多巴胺', '卡酷少儿', '西孟加拉邦', '名列第1', '举行婚礼', '脊椎动物', '开发商', '斯大林格勒', '沃纳·冯·布劳恩', '癌基因', '日本电视台', '维多利亚瀑布', '官僚机构', '日本足球', '机场后勤区', '水力发电', '航天飞机', '军事政变', '市人民政府', '邮政编码', '桂南会战', '詹姆·波特', '日托米尔州', '胚性干细胞', '军政委员会', '非常罕见', '国定古迹', '生活习性', '推古天皇', '中华人民共和国政府', '首次亮相', '国民大会', '公众场合', '国际组织', '平均海拔高度', '卫星电视', '城市轨道交通', '中法战争', '饮食节目', '越南共和国', '民用航空', '大英帝国', '中国基督教', '滨海新区', '中华人民共和国国务院', '神圣罗马帝国皇帝', '加来海峡省', '海洋航空', '新填地街', '旅游景点', '地缘政治', '世界贸易组织', '市区重建', '人民政府', '女子组合', '中性蛋白酶', '马来西亚大选', '新罕布什尔州', '新兴工业化国家', '环境保护', '岛式月台', '南宁轨道交通', '最为严重', '俯式冰橇', '足球协会', '心力衰竭', '陆军官校', '旧中环天星码头', '性别认同', '德国联邦铁路', '火力控制', '新

In [165]:
tmp = random.choice(list(missed_in_terms))
print(tmp)
attn_dict[tmp]

自动售票


[[[13, 16], '0.0335'],
 [[54, 57], '0.0482'],
 [[9, 12], '0.0162'],
 [[33, 36], '0.0133']]

In [ ]:
tmp = random.sample(missed_terms,200)
print(tmp)

In [ ]:
# extract_dict[tmp[0]]

out_terms = set([term[0] for term in out_hl])
lrb_out_cnt = 0
for item in missed_terms:
    if item in out_terms and item not in extract_dict:
        lrb_out_cnt+=1
lrb_out_cnt/len(missed_terms)

In [ ]:
gold_terms = []
with open('/home/wangzhitong/data2/strict_terms.txt','r') as f:
    # gold_terms
    for line in f:
        gold_terms.append(line.strip())
len(gold_terms)

In [ ]:
occur_words =[ term[0]  for sent in tqdm(attn_results) for term in sent]
print(len(occur_words))

In [ ]:

re_patterns = [
    r'^\d+日$',
    r'^\d+月\d*$',
    r'^\d+年$',
    r'^\d+年\d+月$',
    r'^\d+年\d+月\d+日$',
    r'^\d+月\d+日$',
    r'^[公元]*\d+世纪$',
    r'^在\d+年$',
    r'^第\d+$',
    r'^\d+年代$',
    r'^月\d+$'
]
import re
def have_pat(term):
    for pattern in re_patterns:
        if re.search(pattern, term):
            # print('yes')
            return True
    return False
import re
hanzire = re.compile(u'[\u4e00-\u9fa5]')
from collections import Counter
occur_cnt = Counter(occur_words)
gold_terms = [term  for term in gold_terms if term in occur_cnt and occur_cnt[term]>1 and len(term)>2 and not have_pat(term) and hanzire.search(term)]
gold_terms = set(gold_terms)
len(gold_terms)

In [ ]:
sent = list(data[1])
encoded_dict = tk([sent],
                padding='max_length',
                truncation=True,
                max_length=MAX_LENGTH,
                is_split_into_words=True,
                add_special_tokens=True)

input_ids = torch.tensor(encoded_dict['input_ids'], dtype=torch.long)
attention_mask = torch.tensor(encoded_dict['attention_mask'], dtype=torch.long)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
ner_output = ner_model(input_ids, attention_mask=attention_mask,pad_mask = attention_mask)[0].squeeze(0)
# print(ner_output)
sent_len = min(len(sent),MAX_LENGTH-2)
sent_bieos = []
for i in range(1,sent_len+1):
    bieos = ner_output[i]
    bieos = torch.softmax(bieos,dim=-1)
    top2_prob = torch.topk(bieos,2)[1].detach().cpu().numpy()
    top1 = ''
    top2 = ''
    top1 = top2_prob[0]
    prob1 = format(bieos[top1],'.3f')
    sent_bieos.append([id2label[top1]])
    top2 = top2_prob[1]
    prob2 = format(bieos[top2],'.3f')
    thres = 0.6
    if id2label[top1]=='E' and id2label[top2]=='I':
        thres = 0.5
    # else:
    if bieos[top2] / bieos[top1] > thres:
        sent_bieos[-1].append(id2label[top2])
sent_hl = []
for i,label in enumerate(sent_bieos):
    if 'B' in label:
        for j in range(i+1,len(sent_bieos)):
            if 'E' in sent_bieos[j]:
                sent_hl.append([i,j])
                # ner_hl
                break
            elif 'I' in sent_bieos[j]:
                continue
            else:
                break
# ner_hl.append(sent_hl)
# ner_results.append(ner_output.detach().cpu().numpy())
print(sent_hl)
for i,label in enumerate(sent_bieos):
    print(sent[i],ner_output[i+1].detach().cpu().numpy(), label)